1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

In [13]:
import pandas as pd

def get_row_number(data_frame):
    return data_frame.shape[0];

def print_columns(data_frame):
    print ("  --> Columns {}".format(data_frame.columns));

def analyze_class_balance (data_frame,feature_name,target_positive=1,target_negative=0):
    positive_number = get_row_number(data_frame[data_frame[feature_name]==target_positive]);
    negative_number = get_row_number(data_frame[data_frame[feature_name]==target_negative]);
    print ("  --> Found {} p/n: {}/{}".format(feature_name,positive_number,negative_number));
    return (positive_number,negative_number,positive_number+negative_number,feature_name);

def select_columns(data_frame,columns):
    new_data_frame = data_frame[['{}'.format(c) for c in columns]].copy(deep=True);
    return new_data_frame;


In [10]:
data_frame = pd.read_csv("train.csv");


In [11]:
data_frame.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [9]:

# Balance analysis:---------------------------------------
data_frame = pd.read_csv("train.csv")

print_columns(data_frame);
for column in data_frame.columns:
    analyze_class_balance(data_frame,column)


  --> Columns Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')
  --> Found id p/n: 0/0
  --> Found comment_text p/n: 0/0
  --> Found toxic p/n: 15294/144277
  --> Found severe_toxic p/n: 1595/157976
  --> Found obscene p/n: 8449/151122
  --> Found threat p/n: 478/159093
  --> Found insult p/n: 7877/151694
  --> Found identity_hate p/n: 1405/158166


In [17]:
# Case with toxic:----------------------------------------
X = select_columns(data_frame,['comment_text']);
Y = select_columns(data_frame,['toxic'])
(X,Y)


(                                             comment_text
 0       Explanation\nWhy the edits made under my usern...
 1       D'aww! He matches this background colour I'm s...
 2       Hey man, I'm really not trying to edit war. It...
 3       "\nMore\nI can't make any real suggestions on ...
 4       You, sir, are my hero. Any chance you remember...
 ...                                                   ...
 159566  ":::::And for the second time of asking, when ...
 159567  You should be ashamed of yourself \n\nThat is ...
 159568  Spitzer \n\nUmm, theres no actual article for ...
 159569  And it looks like it was actually you who put ...
 159570  "\nAnd ... I really don't think you understand...
 
 [159571 rows x 1 columns],
         toxic
 0           0
 1           0
 2           0
 3           0
 4           0
 ...       ...
 159566      0
 159567      0
 159568      0
 159569      0
 159570      0
 
 [159571 rows x 1 columns])

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

original_comments = [original_comment for original_comment in X['comment_text']]

print("  --> Original comments:\n {}".format(original_comments[:10]));

vectorizer = TfidfVectorizer(max_features=100)
vectorized_comments = vectorizer.fit_transform(original_comments);
for 

print("  --> Vectorized comments:\n{}".format(vectorized_comments[:2]));


# Case with toxic:----------------------------------------

#vectorizer = TfidfVectorizer()

#corpus = v


#corpus = [
#     'This is the first document.',
#     'This document is the second document.',
#     'And this is the third one.',
#     'Is this the first document?',
#]

#vectorizer = TfidfVectorizer()
#X = vectorizer.fit_transform(corpus)

# vectorizer.get_feature_names_out()

#print ("{}".format(X));




  --> Original comments:
 ["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27", "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)", "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.", '"\nMore\nI can\'t make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences fo